In [ ]:
import os

os.chdir("..")
os.getcwd()

In [ ]:
from dataclasses import dataclass
from package.utils import read_yaml


CONFIG = read_yaml("config/config.yaml")

@dataclass
class PredictionConstants:
    ARITFACTS_ROOT_DIR_NAME = CONFIG.ARITFACTS_ROOT_DIR_NAME
    PREDICTION_ROOT_DIR_NAME = CONFIG.PREDICTION.ROOT_DIR_NAME
    OUTPUT_FILE_NAME = CONFIG.PREDICTION.OUTPUT_FILE_NAME

In [ ]:
print("ARITFACTS_ROOT_DIR_NAME:", PredictionConstants.ARITFACTS_ROOT_DIR_NAME)
print("PREDICTION_ROOT_DIR_NAME:", PredictionConstants.PREDICTION_ROOT_DIR_NAME)
print("OUTPUT_FILE_NAME:", PredictionConstants.OUTPUT_FILE_NAME)

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class PredictionConfigEntity:
    ARITFACTS_ROOT_DIR_PATH = Path
    PREDICTION_ROOT_DIR_PATH = Path
    OUTPUT_FILE_PATH = Path

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime
import os


@dataclass
class PredictionConfig:
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    ARITFACTS_ROOT_DIR_PATH = Path(PredictionConstants.ARITFACTS_ROOT_DIR_NAME)
    PREDICTION_ROOT_DIR_PATH = os.path.join(ARITFACTS_ROOT_DIR_PATH, PredictionConstants.PREDICTION_ROOT_DIR_NAME)
    OUTPUT_FILE_PATH = os.path.join(PREDICTION_ROOT_DIR_PATH, f"{timestamp}_{PredictionConstants.OUTPUT_FILE_NAME}")

In [ ]:
print("ARITFACTS_ROOT_DIR_NAME:", PredictionConfig.ARITFACTS_ROOT_DIR_PATH)
print("PREDICTION_ROOT_DIR_PATH:", PredictionConfig.PREDICTION_ROOT_DIR_PATH)
print("OUTPUT_FILE_PATH:", PredictionConfig.OUTPUT_FILE_PATH)

In [ ]:
from dataclasses import dataclass
from package.exception import CustomException
from package.utils import create_dirs, save_json
import sys


@dataclass
class PredictionComponents:
    prediction_config: PredictionConfigEntity

    def predict(self, model, preprocessor, data)->list:
        """makes model's prediction after transforming the data with preprocessor

        Args:
            model (sklearn model): model object for prediction
            preprocessor (): transformation object for data transformation
            data (dataframe): data for prediction

        Returns:
            list: _description_
        """
        try:
            # create required dir's
            create_dirs(self.prediction_config.PREDICTION_ROOT_DIR_PATH)

            # get model and preprocessor
            self.model = model
            self.preprocessor = preprocessor

            # transform data and predict
            data = self.preprocessor.transform(data)
            prediction = self.model.predict(data)

            # save prediction
            save_json({str(self.model).split("(")[0].lower():prediction}, self.prediction_config.OUTPUT_FILE_PATH)
            
            return prediction
        except Exception as e:
            raise CustomException(e, sys)